[View in Colaboratory](https://colab.research.google.com/github/nikhilsingh291/eip_theinkers/blob/master/NIKHIL_BATCH_3_ASSIGNMENT4B.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 32
compression = 0.5
dropout_rate = 0.1

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 21s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 32, dropout_rate = 0.1):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 32, dropout_rate = 0.1):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32
dropout_rate = 0.1
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

Total params: 790,058
Trainable params: 777,994
Non-trainable params: 12,064
__________________________________________________________________________________________________


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,   
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 314s 6ms/step - loss: 1.4220 - acc: 0.4798 - val_loss: 1.6305 - val_acc: 0.4799
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:52 - loss: 1.0347 - acc: 0.6317

50000/50000 [==============================] - 299s 6ms/step - loss: 0.9764 - acc: 0.6536 - val_loss: 1.0712 - val_acc: 0.6463
Epoch 3/50
45568/50000 [==========================>...] - ETA: 24s - loss: 0.8283 - acc: 0.7055

50000/50000 [==============================] - 299s 6ms/step - loss: 0.8248 - acc: 0.7067 - val_loss: 1.7211 - val_acc: 0.5422
Epoch 4/50
50000/50000 [==============================] - 299s 6ms/step - loss: 0.7066 - acc: 0.7506 - val_loss: 1.1414 - val_acc: 0.6626
Epoch 5/50
 3328/50000 [>.............................] - ETA: 4:20 - loss: 0.6497 - acc: 0.7701

50000/50000 [==============================] - 298s 6ms/step - loss: 0.6297 - acc: 0.7787 - val_loss: 0.8843 - val_acc: 0.7116
Epoch 6/50
39424/50000 [======================>.......] - ETA: 59s - loss: 0.5618 - acc: 0.8038

50000/50000 [==============================] - 299s 6ms/step - loss: 0.5640 - acc: 0.8028 - val_loss: 0.9589 - val_acc: 0.7170
Epoch 7/50
50000/50000 [==============================] - 298s 6ms/step - loss: 0.5182 - acc: 0.8183 - val_loss: 0.6306 - val_acc: 0.7860
Epoch 8/50
 1792/50000 [>.............................] - ETA: 4:29 - loss: 0.4660 - acc: 0.8398

...

50000/50000 [==============================] - 301s 6ms/step - loss: 0.0836 - acc: 0.9704 - val_loss: 0.7267 - val_acc: 0.8477
Epoch 45/50
38528/50000 [======================>.......] - ETA: 1:04 - loss: 0.0695 - acc: 0.9747

50000/50000 [==============================] - 302s 6ms/step - loss: 0.0735 - acc: 0.9732 - val_loss: 0.8893 - val_acc: 0.8266
Epoch 46/50
50000/50000 [==============================] - 300s 6ms/step - loss: 0.0800 - acc: 0.9709 - val_loss: 0.7147 - val_acc: 0.8528
Epoch 47/50
 1536/50000 [..............................] - ETA: 4:30 - loss: 0.0616 - acc: 0.9805

50000/50000 [==============================] - 299s 6ms/step - loss: 0.0747 - acc: 0.9724 - val_loss: 0.7727 - val_acc: 0.8485
Epoch 48/50
38528/50000 [======================>.......] - ETA: 1:04 - loss: 0.0722 - acc: 0.9743

50000/50000 [==============================] - 300s 6ms/step - loss: 0.0727 - acc: 0.9738 - val_loss: 0.8171 - val_acc: 0.8400
Epoch 49/50
50000/50000 [==============================] - 299s 6ms/step - loss: 0.0715 - acc: 0.9743 - val_loss: 0.8563 - val_acc: 0.8358
Epoch 50/50
 1536/50000 [..............................] - ETA: 4:30 - loss: 0.0788 - acc: 0.9714

50000/50000 [==============================] - 300s 6ms/step - loss: 0.0722 - acc: 0.9741 - val_loss: 0.6883 - val_acc: 0.8585


In [15]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 26s 3ms/step
Test loss: 0.6883292791604996
Test accuracy: 0.8585


In [13]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')